In [56]:
from sys_data import System_Data
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [57]:
system_data = System_Data(file_name='Case_33BW_Data.xlsx')
model = gp.Model("Expert_Model")

NT, N_Branch, N_TL, N_NL, N_Bus, pIn, N_DG, DG_Mask, R_Branch, X_Branch, Big_M_V, V0, \
    V_min, V_max, Pd, Qd, S_Branch, P_DG_min, P_DG_max, Q_DG_min, Q_DG_max, BigM_SC, BSDG_Mask, \
    Big_M_FF = system_data.args_expert

In [58]:
disturbance = [6,11,29,32]
a = np.ones((system_data.N_NL, system_data.NT)) # 设置普通线路的灾害状态
for dmg in disturbance:
    a[dmg-1, :] = 0

X_tieline0 = np.zeros(system_data.N_TL) # tieline默认一开始都是打开的
Q_svc0 = np.zeros(system_data.N_DG-1) # svc默认输出均为0


In [59]:
PF = model.addMVar(shape=(N_Branch,NT), vtype=GRB.CONTINUOUS, name="PF")
QF = model.addMVar(shape=(N_Branch,NT), vtype=GRB.CONTINUOUS, name="QF")
V = model.addMVar(shape=(N_Bus,NT), vtype=GRB.CONTINUOUS, name="V")

P_dg = model.addMVar(shape=(N_DG,NT), vtype=GRB.CONTINUOUS, name="P_dg")
Q_dg = model.addMVar(shape=(N_DG,NT), vtype=GRB.CONTINUOUS, name="Q_dg")
delta_Qdg = model.addMVar(shape=(N_DG-1,NT-1), vtype=GRB.CONTINUOUS, name="delta_dg")

Pd_rec = model.addMVar(shape=(N_Bus,NT), vtype=GRB.CONTINUOUS, name="Pd_rec")
Qd_rec = model.addMVar(shape=(N_Bus,NT), vtype=GRB.CONTINUOUS, name="Qd_rec")
FF = model.addMVar(shape=(N_Branch,NT), vtype=GRB.CONTINUOUS, name="FF")

X_rec = model.addMVar(shape=(N_Bus,NT), vtype=GRB.BINARY, name="X_rec")
X_EN = model.addMVar(shape=(N_Bus,NT), vtype=GRB.BINARY, name="X_EN")
X_tieline = model.addMVar(shape=(N_TL,NT), vtype=GRB.BINARY, name="X_tieline")
X_line = model.addMVar(shape=(N_NL,NT), vtype=GRB.BINARY, name="X_line")

z_bs = model.addMVar(shape=(N_Bus,NT), vtype=GRB.BINARY, name="z_bs")
b = model.addMVar(shape=(N_Branch,NT), vtype=GRB.BINARY, name="b")
X_BS = model.addMVar(shape=(N_Bus,NT), vtype=GRB.BINARY, name="X_BS")
z_bs1 = model.addMVar(shape=(N_Bus,NT), vtype=GRB.BINARY, name="z_bs1")
z_bs2 = model.addMVar(shape=(N_Bus,NT), vtype=GRB.BINARY, name="z_bs2")
z_dg = model.addMVar(shape=(N_DG-1,NT-1), vtype=GRB.BINARY, name="z_dg")

In [60]:

 # ------------------潮流--------------------
    # 1. Bus PQ Blance: S_jk - S_ij = S_inj
model.addConstr(pIn @ PF == DG_Mask @ P_dg - Pd_rec)
model.addConstr(pIn @ QF == DG_Mask @ Q_dg - Qd_rec)

  # 2. Voltage : U_j - U_i = r*Q_ij + x*P_ij
model.addConstr(pIn.T @ V - R_Branch * PF - X_Branch * QF <= Big_M_V * (1 - b))
model.addConstr(pIn.T @ V - R_Branch * PF - X_Branch * QF >= -Big_M_V * (1 - b))
# model.addConstr(V0 * X_BS + V_min * X_EN - V_min * z_bs <= V)
# model.addConstr(V <= V0 * X_BS + V_max * X_EN - V_max * z_bs)
# model.addConstr(z_bs <= X_BS)
# model.addConstr(z_bs <= X_EN)
# model.addConstr(z_bs >= X_BS + X_EN - 1)

model.addConstr(V >= V_min)
model.addConstr(V <= V_max)
model.addConstr(V[0,:] == V0 )

   # 3. % 3. Load Curtailments
model.addConstr(X_rec <= X_EN)
model.addConstr(X_rec[0,:] == 0)
model.addConstr(Pd_rec == X_rec * Pd)
model.addConstr(Qd_rec == X_rec * Qd)
model.addConstr(X_rec[:,1:] >= X_rec[:,0:-1])
    # % 4. 线路
model.addConstr(PF >= -S_Branch * b)
model.addConstr(PF <= S_Branch * b)
model.addConstr(QF >= -S_Branch * b)
model.addConstr(QF <= S_Branch * b)
  # ------------DG ----------------
model.addConstr(P_dg >= (DG_Mask.T @ X_EN) * P_DG_min)
model.addConstr(P_dg <= (DG_Mask.T @ X_EN) * P_DG_max)
model.addConstr(Q_dg >= (DG_Mask.T @ X_EN) * Q_DG_min)
model.addConstr(Q_dg <= (DG_Mask.T @ X_EN) * Q_DG_max)
# 由于是固定时间断面，针对SVC可能存在多解
# model.addConstr(BigM_SC * (1 - z_dg) <= Q_dg[1:,1:] - Q_dg[1:,0:-1])
# model.addConstr(-BigM_SC * (1 - z_dg) <= delta_Qdg - (Q_dg[1:,1:] - Q_dg[1:,0:-1]) )
# model.addConstr(BigM_SC * (1 - z_dg) >= delta_Qdg - (Q_dg[1:,1:] - Q_dg[1:,0:-1]) )
# model.addConstr(-BigM_SC * z_dg <= delta_Qdg + (Q_dg[1:,1:] - Q_dg[1:,0:-1]) )
# model.addConstr(BigM_SC * z_dg >= delta_Qdg + (Q_dg[1:,1:] - Q_dg[1:,0:-1]) )

 # ---------------Island----------------
    #  1. 一个节点为黑启动节点的条件：存在一个BSDG 
# model.addConstr(X_BS <= BSDG_Mask.sum(axis=1, keepdims=True) * np.ones((1, NT)) )
    # % 2. 每个孤岛是联通的。根据节点是否为黑启动节点，分为两种情况讨论
model.addConstr(pIn @ FF + X_EN <= Big_M_FF * (1 - z_bs1))
model.addConstr(pIn @ FF + X_EN >= -Big_M_FF * (1 - z_bs1))
model.addConstr(z_bs1 - 1 <= X_BS)
model.addConstr(X_BS <= 1 - z_bs1)
model.addConstr(pIn @ FF >= -Big_M_FF * (1 - z_bs2))
model.addConstr(z_bs2 - 1 <= X_BS - 1)
model.addConstr(X_BS - 1 <= 1 - z_bs2)
model.addConstr(X_EN - X_BS >= -Big_M_FF * (1 - z_bs2))
model.addConstr(X_EN - X_BS <= Big_M_FF * (1 - z_bs2))
model.addConstr(z_bs1 + z_bs2 == 1 )

  # % 3. 商品流与线路状态
model.addConstr(-Big_M_FF * b <= FF)
model.addConstr(FF <= Big_M_FF * b)
model.addConstr(b[0:N_NL,:] == X_line) # b=[Xline; Xtieline]为全体线路状态，X_line是变量，由a决定，a是外部输入的普通线路健康状态
model.addConstr(b[N_NL:,:] == X_tieline) # X_tieline是变量，由a决定，a是外部输入的跨区线路健康状态
# model.addConstr(X_line <= a ) #NOTE a 需要在外部输入 这个仅在env.rest才需要


 #  4. 闭合的边数=总节点数-带电孤岛数-不带电孤立节点数
model.addConstr(b.sum(axis=0) == N_Bus - X_BS.sum(axis=0) - (1 - X_EN).sum(axis=0))


    # % 线路操作约束
# model.addConstr(X_tieline[:, 1:] >= X_tieline[:, 0:-1])

# model.addConstr(X_tieline[:, 0] >= X_tieline0) #NOTE X_tieline0 需要在外部输入
# model.addConstr( (X_tieline[:, 1:] - X_tieline[:, 0:-1]).sum(axis=0) <= 1)
# model.addConstr( (X_tieline[:, 0] - X_tieline0).sum(axis=0) <= 1) #NOTE X_tieline0 需要在外部输入

# objective
model.setObjective(-Pd_rec.sum() - 0.01 * X_line.sum() + 0*delta_Qdg.sum(), GRB.MINIMIZE)


In [61]:
model.Params.OutputFlag = True
model.optimize()

X_rec.X

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-1340P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4922 rows, 2528 columns and 12634 nonzeros
Model fingerprint: 0x35503f9c
Variable types: 1144 continuous, 1384 integer (1384 binary)
Coefficient statistics:
  Matrix range     [1e-04, 4e+01]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 4e+01]
Found heuristic solution: objective 0.0000000
Presolve removed 3289 rows and 1373 columns
Presolve time: 0.02s
Presolved: 1633 rows, 1155 columns, 5481 nonzeros
Variable types: 650 continuous, 505 integer (505 binary)
Found heuristic solution: objective -0.1500000

Root relaxation: objective -1.774453e+00, 919 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | In

array([[-0., -0., -0., -0., -0.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [-0., -0., -0., -0., -0.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [-0., -0., -0.,  0., -0.],
       [-0., -0., -0., -0., -0.],
       [-0.,  0.,  0., -0., -0.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [-0., -